In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import json, io, base64, re, os, zipfile
import plotly.graph_objects as go
from IPython.display import display, HTML, clear_output
import plotly.express as px
import ipywidgets as widgets

# Initialize settings
import _settings as settings

# Global variables from settings
plotly_colors = settings.plotly_colors

In [2]:
class DataTransformation:
    def __init__(self):
        self.merged_df = None
        self.group_data = None
        self.output_area = None
        self.merged_uploader = None
        self.group_uploader = None
        self.reset_button = None
        self.callbacks = []  # List to store callbacks
        
    def add_callback(self, callback):
        """Add a callback to be triggered when data changes"""
        self.callbacks.append(callback)
        
    def _trigger_callbacks(self):
        """Trigger all registered callbacks"""
        for callback in self.callbacks:
            callback()
            
    def create_download_link(self, file_path, label):
        """Create a download link for a file."""
        if os.path.exists(file_path):
            with open(file_path, 'rb') as f:
                content = f.read()
            b64_content = base64.b64encode(content).decode('utf-8')

            return widgets.HTML(f"""
                <a download="{os.path.basename(file_path)}" 
                   href="data:application/octet-stream;base64,{b64_content}" 
                   style="color: #0366d6; text-decoration: none; margin-left: 20px; font-size: 14px;">
                    {label}
                </a>
            """)
        else:
            return widgets.HTML(f"""
                <span style="color: red; margin-left: 20px; font-size: 14px;">
                    File "{file_path}" not found!
                </span>
            """)
            
    def setup_data_loading_ui(self):
        """Initialize and display the data loading UI"""
        self.merged_uploader = widgets.FileUpload(
            accept='.csv,.txt,.tsv,.xlsx',
            multiple=False,
            description='Upload Merged Data File',
            layout=widgets.Layout(width='300px'),
            style={'description_width': 'initial'}
        )
        
        self.group_uploader = widgets.FileUpload(
            accept='.json',
            multiple=False,
            description='Upload Group Definition',
            layout=widgets.Layout(width='300px'),
            style={'description_width': 'initial'}
        )

        self.output_area = widgets.Output()

        merged_box = widgets.HBox([
            self.merged_uploader,
            self.create_download_link("example_merged_dataframe.csv", "Example")
        ], layout=widgets.Layout(align_items='center'))

        group_box = widgets.HBox([
            self.group_uploader,
            self.create_download_link("example_group_definition.json", "Example")
        ], layout=widgets.Layout(align_items='center'))
        
        upload_widgets = widgets.VBox([
            widgets.HTML("<h4>Upload Data Files:</h4>"),
            merged_box,
            widgets.HTML("<h4>Upload Group Definition:</h4>"),
            group_box,
            self.output_area
        ])
        
        self.status_area = widgets.Output()
              
        self.merged_uploader.observe(self._on_merged_upload_change, names='value')
        self.group_uploader.observe(self._on_group_upload_change, names='value')

        display(upload_widgets,
                self.status_area)

    def _on_merged_upload_change(self, change):
        if change['type'] == 'change' and change['name'] == 'value':
            with self.output_area:
                self.output_area.clear_output()
                uploaded_files = change.get('new', ())
                if uploaded_files:
                    file_data = uploaded_files[0]
                    self.merged_df = self._load_merged_data(file_data)
                    if self.merged_df is not None:
                        display(HTML(
                            f'<b style="color:green;">Merged data imported: '
                            f'{self.merged_df.shape[0]} rows, {self.merged_df.shape[1]} columns</b>'
                        ))
                        self._trigger_callbacks()

    def _on_group_upload_change(self, change):
        if change['type'] == 'change' and change['name'] == 'value':
            with self.output_area:
                self.output_area.clear_output()
                uploaded_files = change.get('new', ())
                if uploaded_files:
                    file_data = uploaded_files[0]
                    try:
                        content = bytes(file_data.content).decode('utf-8')
                        group_data = json.loads(content)
                        self.group_data = self._process_group_data(group_data)
                        display(HTML(
                            f'<b style="color:green;">Group definition imported successfully with {len(self.group_data)} groups.</b><br>'
                        ))
                        self._trigger_callbacks()
                    except Exception as e:
                        display(HTML(f'<b style="color:red;">Error loading group definition: {str(e)}</b>'))

    def _load_merged_data(self, file_data):
        try:
            content = bytes(file_data.content)
            filename = file_data.name
            extension = filename.split('.')[-1].lower()
            
            file_stream = io.BytesIO(content)
            
            if extension == 'csv':
                df = pd.read_csv(file_stream)
            elif extension in ['txt', 'tsv']:
                df = pd.read_csv(file_stream, delimiter='\t')
            elif extension == 'xlsx':
                df = pd.read_excel(file_stream)
            else:
                raise ValueError("Unsupported file format")
            
            return df
            
        except Exception as e:
            display(HTML(f'<b style="color:red;">Error loading data: {str(e)}</b>'))
            return None

    def _process_group_data(self, json_data):
        """Process and validate the group data structure"""
        try:
            processed_data = {}
            for group_id, group_info in json_data.items():
                if 'grouping_variable' not in group_info:
                    raise ValueError(f"Group {group_id} missing grouping_variable")
                if 'abundance_columns' not in group_info:
                    raise ValueError(f"Group {group_id} missing abundance_columns")

                processed_data[group_id] = {
                    'grouping_variable': group_info['grouping_variable'],
                    'abundance_columns': group_info['abundance_columns']
                }

            return processed_data
        except Exception as e:
            raise ValueError(f"Error processing group data: {str(e)}")

In [3]:
class TotalPeptidePlotter:
    def __init__(self, data_transformer):
        self.data_transformer = data_transformer
        self.plot_output = widgets.Output()
        self.export_output = widgets.Output()
        self.info_output = widgets.Output()
        self.current_fig_abundance = None
        self.current_fig_count = None
        
        # Initialize widgets
        self.setup_widgets()
        
        # Register callback for data changes
        self.data_transformer.add_callback(self._update_group_options)
        
    def setup_widgets(self):
        """Initialize UI widgets"""
        # Color pickers for bars
        self.abundance_color = widgets.ColorPicker(
            description='Abundance Bar Color:',
            value='#0072C6',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.count_color = widgets.ColorPicker(
            description='Count Bar Color:',
            value='#0072C6',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        # Download HTML button
        self.download_html_button = widgets.Button(
            description='Download Interactive Plots',
            button_style='info',
            icon='file',
            layout=widgets.Layout(width='200px'),
            disabled=True
        )
        # Create plot control widgets
        self.plot_button = widgets.Button(
            description='Generate/Update Plots',
            button_style='success',
            icon='refresh',
            layout=widgets.Layout(width='200px')
        )
        
        self.export_button = widgets.Button(
            description='Export Data',
            button_style='info',
            icon='download',
            layout=widgets.Layout(width='200px'),
            disabled=True
        )
        
        # Add group selection widget
        self.group_select = widgets.SelectMultiple(
            description='Groups:',
            options=[],
            layout=widgets.Layout(width='300px', height='100px'),
            style={'description_width': 'initial'}
        )
        
        # Plot type selection
        self.plot_type = widgets.RadioButtons(
            options=['Log Scale', 'Linear Scale'],
            value='Log Scale',
            description='Y-axis Scale:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        # Add label customization
        self.xlabel_widget = widgets.Text(
            description='X Label:',
            placeholder='Enter custom label',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.ylabel_abundance_widget = widgets.Text(
            description='Abundance Y Label:',
            placeholder='Enter custom label',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.ylabel_count_widget = widgets.Text(
            description='Count Y Label:',
            placeholder='Enter custom label',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.title_widget = widgets.Text(
            description='Plot Title:',
            placeholder='Enter custom title',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        # Create layout with sections
        controls_box = widgets.VBox([
            widgets.HTML("<h4>Plot Controls:</h4>"),
            self.group_select,
            self.plot_type
        ])
        
        appearance_box = widgets.VBox([
            widgets.HTML("<h4>Appearance Settings:</h4>"),
            self.xlabel_widget,
            self.ylabel_abundance_widget,
            self.ylabel_count_widget,
            self.title_widget
        ])
        
        # Add color customization section
        color_box = widgets.VBox([
            self.abundance_color,
            self.count_color
        ])
        
        button_box = widgets.VBox([
            widgets.HTML("<h4>Actions:</h4>"),
            widgets.VBox([
                self.plot_button,
                self.export_button,
                self.download_html_button
            ])
        ])
        
        # Create main layout
        self.widget_box = widgets.VBox([
            controls_box,
            appearance_box,
            color_box,
            button_box,
            self.info_output,
            self.plot_output,
            self.export_output
        ])
        
        # Add button callbacks
        self.plot_button.on_click(self._on_plot_button_click)
        self.export_button.on_click(self._on_export_button_click)
        self.download_html_button.on_click(self._on_download_html_click)
        
        # Add observer for data changes
        self.data_transformer.merged_uploader.observe(self._update_group_options, names='value')
    
    def _update_group_options(self, change=None):
        """Update group selection options when data changes"""
        if self.data_transformer.merged_df is not None and self.data_transformer.group_data is not None:
            # Get grouping variables from the group data
            group_options = [
                info['grouping_variable'] 
                for info in self.data_transformer.group_data.values()
            ]
            
            # Update group selection options
            self.group_select.options = group_options
            # Select all groups by default
            self.group_select.value = group_options
                
            
    def _process_total_peptide_data(self):
        """Process peptide data for visualization using group definitions"""
        if self.data_transformer.merged_df is None or self.data_transformer.group_data is None:
            return None
            
        df = self.data_transformer.merged_df.copy()  # Create a copy to avoid modifying original
        results = {}
        
        # Process each group from the group_data dictionary
        for group_id, group_info in self.data_transformer.group_data.items():
            group_name = group_info['grouping_variable']
            abundance_columns = group_info['abundance_columns']
            
            # Calculate total abundance and SEM from the abundance columns
            valid_abundance_cols = [col for col in abundance_columns if col in df.columns]
            
            if not valid_abundance_cols:
                print(f"Warning: No valid abundance columns found for group {group_name}")
                continue
                    
            # Filter for non-zero, non-null values in any abundance column
            temp_df = df[['unique ID'] + valid_abundance_cols].copy()
            
            # Convert abundance columns to numeric, forcing non-numeric values to NaN
            for col in valid_abundance_cols:
                temp_df[col] = pd.to_numeric(temp_df[col], errors='coerce')
            
            temp_df = temp_df[
                temp_df[valid_abundance_cols].notna().any(axis=1) & 
                (temp_df[valid_abundance_cols] != 0).any(axis=1) &
                temp_df['unique ID'].notna()
            ]
            
            if temp_df.empty:
                continue
                    
            # Calculate peptide counts for each replicate
            replicate_counts = []
            for col in valid_abundance_cols:
                count = temp_df[temp_df[col].notna() & (temp_df[col] != 0)]['unique ID'].nunique()
                replicate_counts.append(count)
            
            # Calculate mean count and SEM across replicates
            unique_peptides = np.mean(replicate_counts)
            if len(replicate_counts) > 1:
                count_sem = np.std(replicate_counts, ddof=1) / np.sqrt(len(replicate_counts))
            else:
                count_sem = 0
                
            # Calculate abundance statistics
            abundances = temp_df[valid_abundance_cols].values.astype(float)  # Ensure float type
            peptide_means = np.nanmean(abundances, axis=1)
            total_abundance = np.nansum(peptide_means)
            
            # Calculate SEM for abundance
            peptide_sems = np.nanstd(abundances, axis=1) / np.sqrt(abundances.shape[1])
            total_sem = np.sqrt(np.nansum(peptide_sems ** 2))
            
            results[group_name] = {
                'unique_peptides': unique_peptides,
                'total_Absorbance': total_abundance,
                'total_sem': total_sem,
                'abundance_sem': total_sem,
                'count_sem': count_sem,
                'replicate_data': {
                    'abundance_columns': valid_abundance_cols,
                    'replicate_counts': replicate_counts,
                    'replicate_abundances': [temp_df[col].replace(0, np.nan).sum() for col in valid_abundance_cols]
                }
            }
        
        return results
        
    def plot_total_peptides(self, data):
        """Generate interactive Plotly bar plots for total peptides"""
        if not data:
            return None, None
            
        # Prepare data for plotting
        groups = list(data.keys())
        abundances = [data[group]['total_Absorbance'] for group in groups]
        sems = [data[group]['total_sem'] for group in groups]
        counts = [data[group]['unique_peptides'] for group in groups]
        
        # Create abundance figure with error bars
        fig_abundance = go.Figure()
        fig_abundance.add_trace(go.Bar(
            x=groups,
            y=abundances,
            name='Total Absorbance',
            marker=dict(
                color=self.abundance_color.value,
                line=dict(
                    color='black',
                    width=1
                )
            ),
            error_y=dict(
                type='data',
                array=[data[group]['abundance_sem'] for group in groups],
                visible=True,
                thickness=1.5,
                width=4,
                color='#000000'
            ),
            hovertemplate="Group: %{x}<br>" +
                         "Total Abundance: %{y:.2e}<br>" +
                         "SEM: %{error_y.array:.2e}<br>" +
                         "<extra></extra>"
        ))
        
        # Update abundance layout with black spines
        fig_abundance.update_layout(
            title={
                'text': f"{self.title_widget.value or 'Total Peptide Abundance Distribution'}",
                'y': 0.95,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top',
                'font': {'size': 15, 'color': 'black'}  # Black title text
            },
            xaxis_title=self.xlabel_widget.value or 'Sample Type',
            yaxis_title=self.ylabel_abundance_widget.value or 'Total Abundance (± SEM)',
            template='plotly_white',
            height=700,
            width=1000,
            margin=dict(t=100, l=100, r=100),
            showlegend=False,
            xaxis=dict(
                showline=True,
                linewidth=1,
                linecolor='black',
                mirror=False
            ),
            yaxis=dict(
                showline=True,
                linewidth=1,
                linecolor='black',
                mirror=False
            )
        )
        
        # Update axes for abundance plot
        fig_abundance.update_xaxes(
            tickangle=45,
            title_font={"size": 14},
            tickfont={"size": 12},
            tickfont_color="black",  # Black tick labels
            title_font_color="black",  # Black axis title           
        )
        
        if self.plot_type.value == 'Log Scale':
            fig_abundance.update_yaxes(
                type='log',
                exponentformat='E',
                showexponent='all',
                title_font={"size": 14},
                tickfont={"size": 12},
                tickfont_color="black",  # Black tick labels
                title_font_color="black",  # Black axis title
                gridcolor="lightgray",  # Light gray grid lines
                showgrid=True,  # Show grid lines
                zeroline=False,  # Hide zero line
                tickformat=".1e" 
            )
        else:
            fig_abundance.update_yaxes(
                type='linear',
                title_font={"size": 14},
                tickfont={"size": 12},
                tickfont_color="black",  # Black tick labels
                title_font_color="black",  # Black axis title
                gridcolor="lightgray",  # Light gray grid lines
                showgrid=True,  # Show grid lines
                zeroline=False,  # Hide zero line
            )
        
        # Create count figure with error bars
        fig_count = go.Figure()
        fig_count.add_trace(go.Bar(
            x=groups,
            y=counts,
            name='Peptide Count',
            marker=dict(
                color=self.count_color.value,
                line=dict(
                    color='black',
                    width=1
                )
            ),
            error_y=dict(
                type='data',
                array=[data[group]['count_sem'] for group in groups],
                visible=True,
                thickness=1.5,
                width=4,
                color='#000000'
            ),
                hovertemplate="Group: %{x}<br>" +
                     "Unique Peptides: %{y:.0f}<br>" +  # Changed to .0f for whole numbers
                     "SEM: %{error_y.array:.1f}<br>" +
                     "<extra></extra>"
        ))
        
        # Update count layout with black spines
        fig_count.update_layout(
            title={
                'text': f"{self.title_widget.value or 'Total Peptide Count Distribution'}",
                'y': 0.95,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top',
                'font': {'size': 15, 'color': 'black'}  # Black title text

            },
            
            xaxis_title=self.xlabel_widget.value or 'Sample Type',
            yaxis_title=self.ylabel_count_widget.value or 'Number of Unique Peptides',
            template='plotly_white',
            height=700,
            width=1000,
            margin=dict(t=100, l=100, r=100),
            showlegend=False,
            xaxis=dict(
                showline=True,
                linewidth=1,
                linecolor='black',
                mirror=False
            ),
            yaxis=dict(
                showline=True,
                linewidth=1,
                linecolor='black',
                mirror=False
            )
        )
        
        # Update axes for count plot
        fig_count.update_xaxes(
            tickangle=45,
            title_font={"size": 14},
            tickfont={"size": 12},
            tickfont_color="black",  # Black tick labels
            title_font_color="black",  # Black axis title

        )
        
        fig_count.update_yaxes(
            title_font={"size": 14},
            tickfont={"size": 12},
            tickformat=",d",  # Format with commas for thousands
            tickfont_color="black",  # Black tick labels
            title_font_color="black",  # Black axis title
            gridcolor="lightgray",  # Light gray grid lines
            showgrid=True,  # Show grid lines
            zeroline=False,  # Hide zero line
        )
        
        return fig_abundance, fig_count
    
    def _on_plot_button_click(self, b):
        """Handle plot button click"""
        if not self.group_select.value:
            with self.info_output:
                self.info_output.clear_output(wait=True)
                display(HTML("<b style='color:red'>Please select at least one group.</b>"))
            return
            
        with self.plot_output:
            self.plot_output.clear_output(wait=True)
            
            # Process and plot data
            all_data = self._process_total_peptide_data()
            if all_data:
                # Filter data for selected groups
                selected_groups = list(self.group_select.value)
                filtered_data = {k: v for k, v in all_data.items() 
                               if k in selected_groups}
                
                self.current_fig_abundance, self.current_fig_count = self.plot_total_peptides(
                    filtered_data
                )
                
                if self.current_fig_abundance is not None:
                    self.current_fig_abundance.show()
                    self.current_fig_count.show()
                    self.export_button.disabled = False
                    self.download_html_button.disabled = False
                    
            else:
                print("No data available for plotting.")
                self.export_button.disabled = True
        
    def export_peptide_data(self, data):
        """
        Export peptide data to Excel with summary and replicate details.
        
        Args:
            data (dict): Dictionary containing peptide analysis results
            
        Returns:
            bytes: Excel file content as bytes
        """
        try:
            # Create summary DataFrame
            summary_data = []
            for group, values in data.items():
                summary_data.append({
                    'Group': group,
                    'Total_Absorbance': values['total_Absorbance'],
                    'Abundance_SEM': values['abundance_sem'],
                    'Unique_Peptides': values['unique_peptides'],
                    'Count_SEM': values['count_sem']
                })
            summary_df = pd.DataFrame(summary_data)
            
            # Create replicate details DataFrame
            replicate_data = []
            for group, values in data.items():
                # Get the replicate information
                replicate_info = values['replicate_data']
                
                # Add entry for each replicate
                for i, replicate_name in enumerate(replicate_info['abundance_columns']):
                    replicate_data.append({
                        'Group': group,
                        'Replicate': replicate_name,
                        'Total_Absorbance': replicate_info['replicate_abundances'][i],
                        'Unique_Peptides': replicate_info['replicate_counts'][i]
                    })
            replicate_df = pd.DataFrame(replicate_data)
            
            # Create Excel file in memory
            output = io.BytesIO()
            with pd.ExcelWriter(output, engine='openpyxl') as writer:
                # Write summary sheet
                summary_df.to_excel(
                    writer, 
                    sheet_name='Summary',
                    index=False
                )
                
                # Write replicate details sheet
                replicate_df.to_excel(
                    writer, 
                    sheet_name='Replicate Details',
                    index=False
                )
                
                # Auto-adjust column widths for both sheets
                for sheet in writer.sheets.values():
                    for column in sheet.columns:
                        max_length = 0
                        column = [cell for cell in column if cell.value is not None]
                        for cell in column:
                            try:
                                if len(str(cell.value)) > max_length:
                                    max_length = len(str(cell.value))
                            except:
                                pass
                        adjusted_width = (max_length + 2)
                        sheet.column_dimensions[column[0].column_letter].width = adjusted_width
            
            # Get the Excel file content
            excel_content = output.getvalue()
            output.close()
            
            return excel_content
            
        except Exception as e:
            print(f"Error exporting data: {str(e)}")
            return None
    
    def _on_export_button_click(self, b):
        """Handle export button click event"""
        try:
            # Get the processed data
            data = self._process_total_peptide_data()
            if data is None:
                with self.export_output:
                    self.export_output.clear_output(wait=True)
                    display(HTML('<div style="color: red; padding: 10px;">No data to export.</div>'))
                return
            
            # Export the data
            excel_content = self.export_peptide_data(data)
            if excel_content is None:
                with self.export_output:
                    self.export_output.clear_output(wait=True)
                    display(HTML('<div style="color: red; padding: 10px;">Error creating export file.</div>'))
                return
            
            # Generate timestamp for filename
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f"Total_Peptide_Analysis_{timestamp}.xlsx"
            
            # Create download link
            with self.export_output:
                self.export_output.clear_output(wait=True)
                display(HTML(f'''
                    <div id="export_container_{timestamp}">
                        <a id="export_link_{timestamp}" 
                           href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{base64.b64encode(excel_content).decode()}" 
                           download="{filename}"
                           style="display: none;"></a>
                    </div>
                    <script>
                        document.getElementById('export_link_{timestamp}').click();
                    </script>
                '''))
                
        except Exception as e:
            with self.export_output:
                self.export_output.clear_output(wait=True)
                display(HTML(f'<div style="color: red; padding: 10px;">Error during export: {str(e)}</div>'))
    
    def _on_download_html_click(self, b):
        """Handle HTML plot download"""
        if self.current_fig_abundance is None or self.current_fig_count is None:
            with self.export_output:
                self.export_output.clear_output(wait=True)
                display(HTML("<div style='color: red; padding: 10px;'>No plots available to download.</div>"))
            return
            
        try:
            # Generate timestamp
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            
            # Create HTML wrapper that combines both plots
            combined_html = '''
            <html>
            <head>
                <title>Total Peptide Analysis</title>
                <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
            </head>
            <body>
            '''
            
            # Add abundance plot
            div_id_abundance = f'abundance_chart_{timestamp}'
            combined_html += f'<div id="{div_id_abundance}" style="width: 1000px; height: 700px;"></div>\n'
            combined_html += f'<script>{self.current_fig_abundance.to_json()}</script>\n'
            combined_html += f'''
            <script>
                Plotly.newPlot("{div_id_abundance}", {self.current_fig_abundance.to_json()});
            </script>
            '''
            
            # Add count plot
            div_id_count = f'count_chart_{timestamp}'
            combined_html += f'<div id="{div_id_count}" style="width: 1000px; height: 700px;"></div>\n'
            combined_html += f'<script>{self.current_fig_count.to_json()}</script>\n'
            combined_html += f'''
            <script>
                Plotly.newPlot("{div_id_count}", {self.current_fig_count.to_json()});
            </script>
            '''
            
            combined_html += '</body></html>'
            
            # Create download link
            with self.export_output:
                self.export_output.clear_output(wait=True)
                display(HTML(f'''
                    <div id="download_container_{timestamp}">
                        <a id="download_link_{timestamp}" 
                           href="data:text/html;charset=utf-8;base64,{base64.b64encode(combined_html.encode()).decode()}" 
                           download="total_peptide_plots_{timestamp}.html"
                           style="display: none;"></a>
                        <script>
                            setTimeout(function() {{
                                document.getElementById('download_link_{timestamp}').click();
                            }}, 100);
                        </script>
                    </div>
                '''))
                
        except Exception as e:
            with self.export_output:
                self.export_output.clear_output(wait=True)
                display(HTML(f'<div style="color: red; padding: 10px;">Error downloading plots: {str(e)}</div>'))

    def display(self):
        """Display the total peptide analysis interface"""
        display(self.widget_box)

In [4]:
# Initialize the interface
data_transformer = DataTransformation()
data_transformer.setup_data_loading_ui()

# Create bioactive plotter
peptideplotter = TotalPeptidePlotter(data_transformer)
peptideplotter.display()

Output()

In [5]:
data_transformer.merged_df